# Create supporting graphs and data

_Dataset:_ Supplementary data for Megill and Grewe (2024): "Investigating the limiting aircraft design-dependent and environmental factors of persistent contrail formation".

_Authors:_

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.5194/egusphere-2024-3398

---


### Summary
This notebook creates graphs and data used to support and better understand the analyses in the linked article. Specifically, the following steps are completed:
- We create a water vapour partial pressure vs. temperature plot for a representative aircraft (Figure 1 of linked paper). We use this plot to describe for this aircraft under which ambient conditions a persistent contrail would form, and graphically show each limiting factor. We then find the mixing line slope which intersects with the ice saturation curve and homogeneous freezing temperature. We show that any aircraft with a mixing line slope steeper than this value is no longer limited by the aircraft design and will always produce persistent contrails if the ambient conditions are sufficiently cold and humid.
- We plot the DEPA 2050 progressive global aviation scenario (Supplementary Figure 1), which we use to weight the potential persistent contrail formation values in `41-lm-analyse_limfac.ipynb`.
- To reduce computational time and avoid autocorrelation in the limiting factors analysis, we only use 10 % of all hourly data in the 2010 decade. We visualise which hours were used (Supplementary Figure 2).
- We compare the cumulative distribution functions of ERA5 and MOZAIC/IAGOS (Supplementary Figure 3).
- We visually demonstrate the limiting factors analysis (Figure 2). We also explore where each limiting factor prevents persistent contrail formation for a single time and pressure level.
- Finally, we create the pickled arrays `neighbours`, `perimeters` and `areas`, which are used in the limiting factors analyses. 


### Inputs
- `data/processed/limfac/limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc`: Results of the limiting factors analysis
- `data/external/DEPA2050/FP_con_2050.txt`: DEPA 2050 progressive global aviation scenario in the year 2050. This can be downloaded from https://doi.org/10.5281/zenodo.11442323.
- `data/external/ERA5-IAGOS_QM-CDF_Hofer2024.csv`: Cumulative distribution functions of ERA5 and MOZAIC/IAGOS from Hofer et al. (2024): https://doi.org/10.5194/acp-24-7911-2024. This data can be requested from Sina Hofer, sina.hofer@dlr.de.
- `data/aircraft_specs_v2.nc`: Aircraft specifications created with `02-lm-create_aircraft_specs.ipynb`.
- ERA5 GRIB data: If not performing the study on DKRZ Levante, the ERA5 GRIB data needs to be saved locally and `dir_path` updated. We recommend placing the ERA5 files in `data/raw/`. Ensure Ensure that the file naming matches that of `t_file_path` and `r_file_path`.

### Outputs
- `data/processed/limfac/areas_grib.pickle`: Grid cell areas
- `data/processed/limfac/neighbors_grib.pickle`: Horizontal grid cell neighbors
- `data/processed/limfac/perimeters_grib.pickle`: Horizontal grid cell perimeters
- `data/processed/limfac/vertical_neighbors_grib.pickle`: Vertical grid cell neighbors
- Figures 1 and 2
- Supplementary Figures 1, 2 and 3

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Limiting factors overview

We create a water vapour partial pressure vs. temperature plot for an aircraft with a mixing line slope $G_lim =$ 3.7 Pa/K (Figure 1 of the linked paper). A persistent contrail would form behind this aircraft if the ambient conditions are within the green hashed area. We identify four limiting factors that define this area - droplet formation, droplet freezing, persistence and water supersaturation. We represent each limiting factor by an arrow from a green point (persistent contrail forms) to a red point (persistent contrail does not form). We also show the mixing line slopes for each aircraft type at 250 hPa with dotted lines for reference. CON = conventional (kerosene); HYB = parallel electric hybrid; WET = Water Enhanced Turbofan; H2C = hydrogen combustion; H2FC = hydrogen fuel cell.

To save the figure, uncomment the final line and choose a saving location.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from helper import e_sat_ice, e_sat_water, calc_single_T_max, calc_single_T_min

T_lst = np.arange(200., 246., 0.01)
ppi_sat_lst = e_sat_ice(T_lst)
pplq_sat_lst = e_sat_water(T_lst)

G_lim = 3.7
if G_lim <= 2.0:
    T_max = calc_single_T_max(0, G_lim)
else:
    T_max = calc_single_T_max(1, G_lim)
idx_T_ltmax = (T_lst < T_max) & (T_lst < 235.15)
T_min = calc_single_T_min(T_max, G_lim, e_sat_water(T_max))
G_boundary = G_lim * (T_lst - T_min)


# create figure
fig, ax = plt.subplots()
ax.fill_between(T_lst[idx_T_ltmax], np.maximum(ppi_sat_lst, G_boundary)[idx_T_ltmax], pplq_sat_lst[idx_T_ltmax], color="none", edgecolor="green", hatch="\\\\", alpha=0.3)
ax.axvline(235.15, color="tab:gray", linewidth=1)
ax.plot(T_lst, ppi_sat_lst, color="tab:gray", linewidth=1)
ax.plot(T_lst, pplq_sat_lst, color="tab:gray", linewidth=1)
ax.plot(T_lst, G_boundary)

# plot SAC slopes
G_lst = [0.48, 0.97, 1.15, 1.65, 1.74, 1.93, 4.97, 5.55, 18.6]
T_a = 229.; pp_a = 10.
for G in G_lst:
    ax.plot(np.arange(T_a, 246.), G * (np.arange(T_a, 246.) - T_a) + pp_a, "k--", lw=0.8, alpha=0.3)

# plot limiting factors
temp1 = (234.8, 22.4)
temp2 = (235.6, 22.4)
gbc1 = (233.5, 15.9)
gbc2 = (234.7, 15.9)
ice1 = (229, 10)
ice2 = (229, 5)
cir1 = (225, 6.5)
cir2 = (225, 11)
points1 = [temp1, gbc1, ice1, cir1]
points2 = [temp2, gbc2, ice2, cir2]
arrow_properties = dict(arrowstyle='->', color='k')
for point1, point2 in zip(points1, points2):
    ax.plot(*point1, 'go')
    ax.plot(*point2, 'ro')
    ax.annotate('', xy=point2, xytext=point1, arrowprops=arrow_properties)

# add ranges for different fuels
bar_properties = dict(arrowstyle='|-|', color='k', linewidth=0.5, mutation_scale=4)
ax.annotate('H$_2$FC', xy=(234.5, 39), xytext=(230.3, 39), arrowprops=bar_properties, va="center", ha="right")  # H2FC
ax.annotate('H$_2$C', xy=(235, 38), xytext=(233.5, 38), arrowprops=bar_properties, va="center", ha="right")  # H2C
ax.annotate('CON', xytext=(239.4, 31.5), xy=(239.4, 26.8), arrowprops=bar_properties, va="bottom", ha="center")  # JA1
ax.annotate('', xy=(238.7, 27.8), xytext=(238.7, 20.8), arrowprops=bar_properties, va="top", ha="center")  # hybrid
ax.annotate('HYB', xy=(239.4, 24.3), va="center", ha="center")
ax.annotate('WET', xy=(239.4, 20.8), xytext=(239.4, 14.4), arrowprops=bar_properties, va="top", ha="center")  # WET

# add labels
ax.annotate("water sat.", xy=(220.3, 6), rotation=13., color="tab:gray")
ax.annotate("ice sat.", xy=(220.5, 1), rotation=6., color="tab:gray")
ax.annotate("droplet\nfreezing", xy=(temp2[0] + 0.6, temp2[1] - 1.3))
ax.annotate("droplet\nformation", xy=(gbc2[0] + 1, gbc2[1] - 1.3))
ax.annotate("persistence", xy=(ice2[0], ice2[1] - 2.5), ha="center")
ax.annotate("water\nsupersaturation", xy=(cir2[0], cir2[1] + 2), ha="center")
ax.annotate(f"$G$ = {G_lim} Pa/K", xy=(237, 30.3), rotation=53., color="tab:blue")

# set axis labels
ax.set_xlim([220, 240])
ax.set_ylim([0, 40])
ax.set_xlabel("Temperature [K]")
ax.set_ylabel("Water vapour partial pressure [Pa]")

fig.tight_layout()
# fig.savefig("figs/limfac_overview.pdf")

Next, we wish to find the mixing line slope $G$ above which the aircraft design is no longer limiting. This corresponds to the slope for which the limiting, blue line intersects the ice saturation curve at the homogeneous freezing temperature (235.15 K). At this and steeper slopes, the shape of the green hashed area would not be influenced by the blue line. Therefore, whether a persistent contrail forms depends only on the ambient conditions.

We find this limiting slope to be 4.29 Pa/K, with the ice saturation curve calculated using Tetens' formula (also used in ERA5). Hydrogen combustion aircraft have a mixing line slope lower than 4.29 Pa/K at altitudes higher than 217 hPa; high-voltage hydrogen fuel cell aircraft above 205 hPa; and low-voltage hydrogen fuel cell aircraft at all conventional altitudes. Hydrogen fuel cell aircraft are not likely to fly at these kinds of altitudes. Therefore, for most intents and purposes, for hydrogen-powered aircraft whether a persistent contrail forms is only dependent on the ambient conditions.

In [ ]:
# calculate G above which aircraft design is no longer limiting
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton
from helper import e_sat_ice, e_sat_water, e_sat_water_prime

# define saturation lines
T_lst = np.arange(200., 246., 0.01)
ppi_sat_lst = e_sat_ice(T_lst)
pplq_sat_lst = e_sat_water(T_lst)

# calculate G_lim that crosses (235.15, e_sat_ice(235.15))
def fmin_T_max(T_max, T_a, p_a):
    return p_a - e_sat_water(T_max) + e_sat_water_prime(T_max) * (T_max - T_a)

T = 235.15
T_max = newton(fmin_T_max, T+10., args=(T, e_sat_ice(T)))
G_lim = (e_sat_water(T_max) - e_sat_ice(T)) / (T_max - T)
print(f"G_lim = {G_lim}")

# calculate G boundary
if G_lim <= 2.0:
    T_max = calc_single_T_max(0, G_lim)
else:
    T_max = calc_single_T_max(1, G_lim)
idx_T_ltmax = (T_lst < T_max) & (T_lst < 235.15)
T_min = calc_single_T_min(T_max, G_lim, e_sat_water(T_max))
G_boundary = G_lim * (T_lst - T_min)

# draw figure
fig, ax = plt.subplots()
ax.fill_between(T_lst[idx_T_ltmax], np.maximum(ppi_sat_lst, G_boundary)[idx_T_ltmax], pplq_sat_lst[idx_T_ltmax], color="none", edgecolor="green", hatch="\\\\", alpha=0.3)
ax.axvline(235.15, color="tab:gray", linewidth=1)
ax.plot(T_lst, ppi_sat_lst, color="tab:gray", linewidth=1)
ax.plot(T_lst, pplq_sat_lst, color="tab:gray", linewidth=1)
ax.plot(T_lst, G_boundary)
ax.annotate(f"$G$ = {G_lim:.2f} Pa/K", xy=(238.5, 34.5), rotation=59., color="tab:blue")

# set axis labels
ax.set_xlim([220, 245])
ax.set_ylim([0, 45])
ax.set_xlabel("Temperature [K]")
ax.set_ylabel("Water vapour partial pressure [Pa]")

fig.tight_layout()

## DEPA 2050

Here, we show the DEPA 2050 progressive global aviation scenario used to weight the results in the linked paper (Supplementary Figure 1). The dataset can be downloaded here: https://doi.org/10.5281/zenodo.11442323. We also show the ERA5 pressure levels considered in this study.

To run, set the top-level project directory `project_dir`. To save the figure, uncomment the final line and choose a saving location.

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

project_dir = ""  # set top-level project directory
processed_data_dir = project_dir + "data/processed/limfac/"

# loading data
ds = xr.open_dataset(f"{processed_data_dir}limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc")
depa_dir = project_dir + "data/external/DEPA2050/"
df_depa = pd.read_csv(f"{depa_dir}FP_con_2050.txt", delim_whitespace=True, header=None,
                      names=["longitude", "latitude", "level", "fuel", "NOx", "dist", "temp"])

# plot DEPA2050 data
df_depa2 = df_depa.drop(columns=["NOx", "fuel", "temp"])
df_summed = df_depa2.groupby(['latitude', 'level'], as_index=False)['dist'].sum()
fuel_pivot = df_summed.pivot(index='level', columns='latitude', values='dist')

# Create a meshgrid for plotting
latitudes = fuel_pivot.columns.values
levels = fuel_pivot.index.values
fuel_values = fuel_pivot.values / 1e8  # km -> 1e8 km for labelling

# Plotting using contourf
fig, ax = plt.subplots(figsize=(7, 5))
contour = ax.contourf(latitudes, levels, fuel_values, cmap='Oranges')
fig.colorbar(contour, label='Yearly distance flown [10$^8$ km]')

era5_levels = [150, 175, 200, 225, 250, 300, 350]
level_labels = [150, 175, 200, 225, 250, 300, 350, 400, 500, 600, 700, 800, 900, 1000]
ax.set_yticks(level_labels)
for lvl in era5_levels:
    ax.axhline(lvl, ls="--", lw=1, c="gray")

ax.set_xlim([-90, 90])
ax.set_ylim([125, 1013])
ax.set_xlabel('Latitude [deg]', fontsize=12)
ax.set_ylabel('Pressure level [hPa]', fontsize=12)
plt.gca().invert_yaxis()  # Invert y-axis to have higher levels at the top
ax.text(-85, 390, "ERA5 levels", color="gray")

fig.tight_layout()
# fig.savefig("figs/DEPA2050_contour.pdf")

## Selected hours

To limit computational time and avoid autocorrelation, in our analysis on the limiting factors we randomly select 10% of the results in the 2010 decade. This corresponds to 2160 hours per season, which we define as DJF, MAM, JJA and SON. Here, we visualise (Supplementary Figure 2) the hours that were selected using the `random_seed` of 42 (the Answer to the Ultimate Question of Life, the Universe, and Everything). To save the figure, uncomment the final line and choose a saving location.

In [ ]:
# set the seed and calculate selected hours
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from helper import select_random_hours

random_seed = 42  # seed for np.random
num_h_per_s = 2160  # total number of random hours per season (DJF, MAM, JJA, SON)
selected_hours = select_random_hours(num_h_per_s, seed=random_seed) 

# show selected hours
df_selected_hours = pd.DataFrame(np.concatenate(selected_hours), columns=["Date"])
df_selected_hours['Month'] = df_selected_hours['Date'].dt.to_period('M')
monthly_counts = df_selected_hours.groupby('Month').size().reset_index(name='Counts')
monthly_counts['Year'] = monthly_counts['Month'].dt.year
monthly_counts['Month_of_Year'] = monthly_counts['Month'].dt.month
heatmap_data = monthly_counts.pivot_table(index='Year', columns='Month_of_Year', values='Counts')

# plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap='coolwarm', linewidths=.5, vmin=0, vmax=100,
            cbar_kws={'label': 'Number of selected hours per month'})
plt.xlabel('Month', fontsize=12)
plt.ylabel('Year', fontsize=12)
plt.xticks(ticks=np.arange(0.5, 12.5, 1), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.tight_layout()
# plt.savefig("figs/selected_hours_per_month.pdf")

## Comparison of ERA5 with MOZAIC/IAGOS

Numerous authors have shown the difficulty of estimating ice supersaturated regions (ISSRs) and hence contrail persistence with reanalysis data. There have thus been many suggestions on how to correct ERA5 data to more accurately estimate ISSRs, both at a regional and global scale. In this study, we enhance the ERA5 humidity by applying a factor of 1/RHi_C for RHi_C = [1.0, 0.98, 0.95, 0.90]. This has various limitations, as explained in more detail in the linked article. Here, we compare the cumulative distribution functions of ERA5 and MOZAIC/IAGOS from Hofer et al. (2024): https://doi.org/10.5194/acp-24-7911-2024. We find that RHi_C = 0.95 provides a good fit against MOZAIC/IAGOS data for RHi <= 1.0, which is the area that we are interested in (Supplementary Figure 3).

To run, set the top-level project directory `project_dir`. To save the figure, uncomment the final line and choose a saving location.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

project_dir = ""  # set top-level project directory
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

cdf_data = np.loadtxt(f"{project_dir}data/external/ERA5-IAGOS_QM-CDF_Hofer2024.csv", delimiter=",", skiprows=1)

cdf_x_ERA5 = cdf_data[:, 0]
cdf_y_ERA5 = cdf_data[:, 1]
cdf_x_MI = cdf_data[:, 2]
cdf_y_MI = cdf_data[:, 3]

fig, ax = plt.subplots()
ax.plot(cdf_x_MI, cdf_y_MI, c=colors[0], lw=3, label="MOZAIC/IAGOS (Hofer et al., 2024b)")
ax.plot(cdf_x_ERA5, cdf_y_ERA5, c=colors[1], label="ERA5 (Hofer et al., 2024b)")
ax.plot(cdf_x_ERA5 / 0.98, cdf_y_ERA5, c=colors[2], ls="--", label="ERA5 RHi$_C$ = 0.98")
ax.plot(cdf_x_ERA5 / 0.95, cdf_y_ERA5, c=colors[3], ls="--", label="ERA5 RHi$_C$ = 0.95")
ax.plot(cdf_x_ERA5 / 0.90, cdf_y_ERA5, c=colors[4], ls="--", label="ERA5 RHi$_C$ = 0.90")
ax.axvline(1.0, c="tab:gray", lw=1)
ax.legend(loc="best", framealpha=1)

# axis definitions
ax.set_xlim([0.5, 1.4])
ax.set_xlabel("RHi [-]", fontsize=12)
ax.set_ylim([0.5, 1.05])
ax.set_ylabel("CDF [-]", fontsize=12)

fig.tight_layout()
# fig.savefig("figs/RHi-CDF_ERA5-MI_modHofer2024.pdf")

## How does the analysis of persistent contrail formation region boundaries work?

The objective of this analysis (described in Section 2.6 of the linked paper) is to understand the relative importance of the limiting factors in defining the horizontal and vertical boundaries of persistent contrail formation regions. We demonstrate the analysis graphically using examples over The Netherlands and Germany for a single time and pressure level. For a given aircraft design, green cloud icons represent grid points where a persistent contrail would form; black points where none are expected. The borders around grid cells (solid black lines) are calculated using the `scipy.spatial.SphericalVoronoi` method. Cell neighbours (gray dotted lines) are found by determining which cells share a pair of edge vertices. We are interested in the boundaries of persistent contrail formation regions, therefore we need to identify neighbouring cells with different icons (red, solid lines). Two adjacent cells $i$ and $j$ share a border $d_{ij}$ (thick green line), which we calculate using the Haversine formula assuming an Earth radius of 6371 km. The colour gradient bar indicates the length of the border around each grid cell.

The main values that can be changed are:
- `ac_id`: the aircraft identifier. In the example, we use a hydrogen combustion aircraft AC4.
- `date_str`: date to analyse, format YYYY-MM-DD. In the example, we use 2024-02-11.
- `i_time`: hour in the day (UTC). In the example, we use 8.
- `i_lvl`: index of the pressure level used. In the example, we use 1, corresponding to 300 hPa.
- `extent`: extent of latitude-longitude in the plot, format [min_lon, max_lon, min_lat, max_lat]. In the examples here, we use the Netherlands [3.0, 7.5, 50.5, 53.8] and Germany [5.5, 15.5, 47., 55.].
- `limfac`: choice of limiting factor to use. Choices are: "limfac_tot", "limfac_per", "limfac_frz".

To run, set the top-level project directory `project_dir` and, if not running the analysis on DKRZ Levante, set `dir_path` to the location of the ERA5 GRIB files. We recommend placing these files in `data/raw/`. Ensure that the file naming matches that of `t_file_path` and `r_file_path` in ln. 27 and 28. To save the figures, uncomment the respective final lines and choose a saving location.

In [ ]:
import xarray as xr
import numpy as np
import cf2cdm
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize
from scipy.spatial import SphericalVoronoi
import warnings
import pickle
from helper import lonlat_to_xyz, xyz_to_lonlat, limfac_example

# define directories
project_dir = ""  # set top-level project directory
processed_data_dir = project_dir + "data/processed/limfac/"

# load aircraft 
ac_id = "AC4"
ac = xr.load_dataset(project_dir+"data/aircraft_specs_v2.nc").sel(id=ac_id)

# load GRIB file
dir_path = "/pool/data/ERA5/E5/pl/an/1H/"
date_str = "2024-02-11"  # random date for example
t_file_path = f"{dir_path}130/E5pl00_1H_{date_str}_130.grb"  # temperature file (130)
r_file_path = f"{dir_path}157/E5pl00_1H_{date_str}_157.grb"  # relative humidity file (157)
dsg_t = xr.open_dataset(t_file_path, engine="cfgrib", backend_kwargs={"indexpath": None})
dsg_r = xr.open_dataset(r_file_path, engine="cfgrib", backend_kwargs={"indexpath": None})
dsg = xr.merge([dsg_t, dsg_r])
with warnings.catch_warnings():  # ignoring UserWarning from cf2cdm when converting coordinate time -> time
    warnings.simplefilter('ignore')
    dsg = cf2cdm.translate_coords(dsg, cf2cdm.ECMWF)  # convert to ECMWF coordinates
dsg = dsg.isel(level=[18, 19, 20, 21, 22, 23, 24])  # selecting only the levels that are interesting

# load neighbors and perimeters
with open(processed_data_dir+"neighbors_grib.pickle", "rb") as f:
    neighbors = pickle.load(f)
with open(processed_data_dir+"perimeters_grib.pickle", "rb") as f:
    perimeters_dict = pickle.load(f)
perimeters = np.array(list(perimeters_dict.values()))  # convert to 1D array for normalisation

# define lat-lon points
lat = dsg.latitude.values
lon = dsg.longitude.values
points = np.empty((len(lat), 2))
points[:, 0] = lon
points[:, 1] = lat

# calculate spherical Voronoi
points_cartesian = lonlat_to_xyz(lon, lat, radians=False)
sv = SphericalVoronoi(points_cartesian)
sv.sort_vertices_of_regions()  # this ensures that the vertices can be plotted properly

# calculate limfacs
i_lvl = 1; lvl = dsg.level.values[i_lvl]
i_time = 8
dsg_1t = dsg.isel(level=i_lvl, time=i_time)
limfac_tot = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "limfac_tot") * perimeters
limfac_per = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "limfac_per") * perimeters
limfac_frz = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "limfac_frz") * perimeters
mask = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "cont_bool")

In [ ]:
# NETHERLANDS example plot

# choose which limiting factor to show
limfac = limfac_tot

# set extent
extent = [3.0, 7.5, 50.5, 53.8]  # NL
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

# define figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# plot geographical features
# code adapted from: https://stackoverflow.com/questions/62308857/borders-and-coastlines-interfering-in-python-cartopy
resol = '50m'  # use data at this scale
bodr = cartopy.feature.NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land', scale=resol, facecolor='none', alpha=0.7)
land = cartopy.feature.NaturalEarthFeature('physical', 'land', \
    scale=resol, edgecolor='k', facecolor=cfeature.COLORS['land'])
ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', \
    scale=resol, edgecolor='none', facecolor=cfeature.COLORS['water'])
lakes = cartopy.feature.NaturalEarthFeature('physical', 'lakes', \
    scale=resol, edgecolor='b', facecolor=cfeature.COLORS['water'])
rivers = cartopy.feature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', \
    scale=resol, edgecolor='b', facecolor='none')

ax.add_feature(land, facecolor='beige', zorder=0)
ax.add_feature(ocean, linewidth=0.2, zorder=0)
ax.add_feature(lakes, zorder=0)
ax.add_feature(rivers, linewidth=0.5, zorder=0)
ax.add_feature(bodr, linestyle='--', edgecolor='k', alpha=1, zorder=1)


# calculate which points are within the extent
pts_ll = xyz_to_lonlat(sv.points, radians=False)
vts_ll = xyz_to_lonlat(sv.vertices, radians=False)
pts_ll_ext = []
idx_ext_pts = []
for i_pt, pt in enumerate(pts_ll):
    if extent[2]-1 <= pt[1] <= extent[3]+1 and extent[0]-1 <= pt[0] <= extent[1]+1:
        pts_ll_ext.append(pt)
        idx_ext_pts.append(i_pt)
pts_ll_ext = np.array(pts_ll_ext)


# set colours
cmap = get_cmap("Greens")
norm = Normalize(vmin=min(limfac), vmax=max(limfac))

# plot regions and add background green colour if persistent contrails form
for i_reg, region in enumerate(sv.regions):
    if -1 not in region and len(region) > 0:
        polygon = [vts_ll[n] for n in region]
        polygon.append(vts_ll[region[0]])  # close the cell
        if any(extent[2]-1 <= v[1] <= extent[3]+1 and extent[0]-1 <= v[0] <= extent[1]+1 for v in polygon):
            if mask[i_reg]:
                fc = cmap(norm(limfac[i_reg]))
                poly = mpatches.Polygon(polygon, closed=True, fill=True, fc=fc, alpha=0.8, transform=ccrs.PlateCarree(), zorder=2)
                ax.add_patch(poly)
            ax.plot(*zip(*polygon), "k", lw=1., transform=ccrs.PlateCarree(), zorder=3)


# plot edges used in summation
for i in idx_ext_pts:
    for j in neighbors[i]:
        if j < i and mask[i] != mask[j]:
            v1, v2 = neighbors[i][j]['v1'], neighbors[i][j]['v2']
            start_vertex = vts_ll[v1]
            end_vertex = vts_ll[v2]
            ax.plot([start_vertex[0], end_vertex[0]], [start_vertex[1], end_vertex[1]], 'g-', lw=4., transform=ccrs.Geodetic(), zorder=5)


# plot points and neighbours
for n1 in idx_ext_pts:
    pt1 = points[n1]
    if mask[n1]:
        m, ms, c = "$\u2601$", 12, "g"
    else:
        m, ms, c = "o", 5, "k"
    ax.plot(pt1[0], pt1[1], marker=m, c=c, markersize=ms, transform=ccrs.Geodetic(), zorder=6)
    for n2 in neighbors[n1]:
        if n2 < n1:
            pt2 = points[n2]
            if mask[n1] != mask[n2]:
                c, ls, lw = "r", "-", 1.4
            else:
                c, ls, lw = "gray", ":", 1.
            ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], c=c, ls=ls, lw=lw, transform=ccrs.Geodetic(), zorder=4)


sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.8)
cbar.set_label('Total limfac edge length [km]')

fig.tight_layout()
# fig.savefig("figs/working_example_NL.png", dpi=600)

In [ ]:
# GERMANY example plot

# set options
limfac = limfac_per

# set extent
extent = [5.5, 15.5, 47., 55.]  # germany
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

# define figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# plot geographical features
# code adapted from: https://stackoverflow.com/questions/62308857/borders-and-coastlines-interfering-in-python-cartopy
resol = '50m'  # use data at this scale
bodr = cartopy.feature.NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land', scale=resol, facecolor='none', alpha=0.7)
land = cartopy.feature.NaturalEarthFeature('physical', 'land', \
    scale=resol, edgecolor='k', facecolor=cfeature.COLORS['land'])
ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', \
    scale=resol, edgecolor='none', facecolor=cfeature.COLORS['water'])
lakes = cartopy.feature.NaturalEarthFeature('physical', 'lakes', \
    scale=resol, edgecolor='b', facecolor=cfeature.COLORS['water'])
rivers = cartopy.feature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', \
    scale=resol, edgecolor='b', facecolor='none')

ax.add_feature(land, facecolor='beige', zorder=0)
ax.add_feature(ocean, linewidth=0.2, zorder=0)
ax.add_feature(lakes, zorder=0)
ax.add_feature(rivers, linewidth=0.5, zorder=0)
ax.add_feature(bodr, linestyle='--', edgecolor='k', alpha=1, zorder=1)


# calculate which points are within the extent
pts_ll = xyz_to_lonlat(sv.points, radians=False)
vts_ll = xyz_to_lonlat(sv.vertices, radians=False)
pts_ll_ext = []
idx_ext_pts = []
for i_pt, pt in enumerate(pts_ll):
    if extent[2]-1 <= pt[1] <= extent[3]+1 and extent[0]-1 <= pt[0] <= extent[1]+1:
        pts_ll_ext.append(pt)
        idx_ext_pts.append(i_pt)
pts_ll_ext = np.array(pts_ll_ext)


# set colours
cmap = get_cmap("Greens")
norm = Normalize(vmin=min(limfac), vmax=max(limfac))

# plot regions and add background green colour if persistent contrails form
for i_reg, region in enumerate(sv.regions):
    if -1 not in region and len(region) > 0:
        polygon = [vts_ll[n] for n in region]
        polygon.append(vts_ll[region[0]])  # close the cell
        if any(extent[2]-1 <= v[1] <= extent[3]+1 and extent[0]-1 <= v[0] <= extent[1]+1 for v in polygon):
            if mask[i_reg]:
                fc = cmap(norm(limfac[i_reg]))
                poly = mpatches.Polygon(polygon, closed=True, fill=True, fc=fc, alpha=0.8, transform=ccrs.PlateCarree(), zorder=2)
                ax.add_patch(poly)
            ax.plot(*zip(*polygon), "k", lw=1., transform=ccrs.PlateCarree(), zorder=3)


# plot edges used in summation
for i in idx_ext_pts:
    for j in neighbors[i]:
        if j < i and mask[i] != mask[j]:
            v1, v2 = neighbors[i][j]['v1'], neighbors[i][j]['v2']
            start_vertex = vts_ll[v1]
            end_vertex = vts_ll[v2]
            ax.plot([start_vertex[0], end_vertex[0]], [start_vertex[1], end_vertex[1]], 'g-', lw=4., transform=ccrs.Geodetic(), zorder=5)


# plot points and neighbours
for n1 in idx_ext_pts:
    pt1 = points[n1]
    if mask[n1]:
        m, ms, c = "$\u2601$", 8, "g"
    else:
        m, ms, c = "o", 3, "k"
    ax.plot(pt1[0], pt1[1], marker=m, c=c, markersize=ms, transform=ccrs.Geodetic(), zorder=6)
    for n2 in neighbors[n1]:
        if n2 < n1:
            pt2 = points[n2]
            if mask[n1] != mask[n2]:
                c, ls, lw = "r", "-", 1.4
            else:
                c, ls, lw = "gray", ":", 1.
            ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], c=c, ls=ls, lw=lw, transform=ccrs.Geodetic(), zorder=4)


sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.8)
cbar.set_label('Total edge length [km]')

fig.tight_layout()
# fig.savefig("figs/working_example_DE.png", dpi=600)

Now we wish to understand where each factor limits persistent contrail formation using a single time and pressure level. In the following analysis, we plot where each factor allows persistent contrail formation and show the potential persistent contrail formation for four different aircraft designs. We see that droplet formation sometimes limits persistent contrail formation for aircraft with low mixing line slopes (e.g. WET-75), but mostly the most limiting factor is persistence (ice supersaturation).

To run, set the top-level project directory `project_dir` and, if not running the analysis on DKRZ Levante, set `dir_path` to the location of the ERA5 GRIB files. We recommend placing these files in `data/raw/`. Ensure that the file naming matches that of `t_file_path` and `r_file_path` in ln. 27 and 28.

In [ ]:
import xarray as xr
import numpy as np
import cf2cdm
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize
from scipy.spatial import SphericalVoronoi
import warnings
import pickle
from helper import lonlat_to_xyz, xyz_to_lonlat, limfac_example


# define directories
project_dir = ""  # set top-level directory location
processed_data_dir = project_dir + "data/processed/limfac/"

# load GRIB file
dir_path = "/pool/data/ERA5/E5/pl/an/1H/"
date_str = "2024-02-11"  # random date for example
t_file_path = f"{dir_path}130/E5pl00_1H_{date_str}_130.grb"  # temperature file (130)
r_file_path = f"{dir_path}157/E5pl00_1H_{date_str}_157.grb"  # relative humidity file (157)
dsg_t = xr.open_dataset(t_file_path, engine="cfgrib", backend_kwargs={"indexpath": None})
dsg_r = xr.open_dataset(r_file_path, engine="cfgrib", backend_kwargs={"indexpath": None})
dsg = xr.merge([dsg_t, dsg_r])
with warnings.catch_warnings():  # ignoring UserWarning from cf2cdm when converting coordinate time -> time
    warnings.simplefilter('ignore')
    dsg = cf2cdm.translate_coords(dsg, cf2cdm.ECMWF)  # convert to ECMWF coordinates
dsg = dsg.isel(level=[18, 19, 20, 21, 22, 23, 24])  # selecting only the levels that are interesting

# load neighbors and perimeters
with open(processed_data_dir+"neighbors_grib.pickle", "rb") as f:
    neighbors = pickle.load(f)
with open(processed_data_dir+"perimeters_grib.pickle", "rb") as f:
    perimeters_dict = pickle.load(f)
perimeters = np.array(list(perimeters_dict.values()))  # convert to 1D array for normalisation

# define lat-lon points
lat = dsg.latitude.values
lon = dsg.longitude.values
points = np.empty((len(lat), 2))
points[:, 0] = lon
points[:, 1] = lat

# calculate spherical Voronoi
points_cartesian = lonlat_to_xyz(lon, lat, radians=False)
sv = SphericalVoronoi(points_cartesian)
sv.sort_vertices_of_regions()  # this ensures that the vertices can be plotted properly

# define level and time to analyse  (good one so far: lvl 2, time 17 on Feb 11th 2024)
i_lvl = 2; lvl = dsg.level.values[i_lvl]
i_time = 17
dsg_1t = dsg.isel(level=i_lvl, time=i_time)

# load aircraft 
ac_ids = ["AC8", "AC7", "AC0", "AC4"]
ac_colors = ['#2271B2', '#3DB7E9', '#359B73', '#E69F00']

# calculate limfacs
mask_arr = np.empty((len(ac_ids), 542080))
slope_arr = np.empty((len(ac_ids), 542080))

for i_ac, ac_id in enumerate(ac_ids):
    ac = xr.load_dataset(project_dir+"data/aircraft_specs_v2.nc").sel(id=ac_id)
    mask_arr[i_ac, :] = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "cont_bool")
    slope_arr[i_ac, :] = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "slope_bool")

issr_arr = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "issr_bool")
temp_arr = limfac_example(dsg_1t, lvl, ac, neighbors, perimeters, "temp_bool")

In [ ]:
# GERMANY example plot

# set options
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

# set extent
extent = [5.5, 15.5, 47., 55.]  # germany
# extent = [20.0, 32.5, 59.5, 70.5]  # finland
# extent = [2., 18., 45., 56.]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

# define figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# plot geographical features
# code adapted from: https://stackoverflow.com/questions/62308857/borders-and-coastlines-interfering-in-python-cartopy
resol = '50m'  # use data at this scale
bodr = cartopy.feature.NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land', scale=resol, facecolor='none', alpha=0.7)
land = cartopy.feature.NaturalEarthFeature('physical', 'land', \
    scale=resol, edgecolor='k', facecolor=cfeature.COLORS['land'])
ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', \
    scale=resol, edgecolor='none', facecolor=cfeature.COLORS['water'])
lakes = cartopy.feature.NaturalEarthFeature('physical', 'lakes', \
    scale=resol, edgecolor='b', facecolor=cfeature.COLORS['water'])
rivers = cartopy.feature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', \
    scale=resol, edgecolor='b', facecolor='none')

ax.add_feature(land, facecolor='beige', zorder=0)
ax.add_feature(ocean, linewidth=0.2, zorder=0)
ax.add_feature(lakes, zorder=0)
ax.add_feature(rivers, linewidth=0.5, zorder=0)
ax.add_feature(bodr, linestyle='--', edgecolor='k', alpha=1, zorder=1)


# calculate which points are within the extent
pts_ll = xyz_to_lonlat(sv.points, radians=False)
vts_ll = xyz_to_lonlat(sv.vertices, radians=False)
pts_ll_ext = []
idx_ext_pts = []
for i_pt, pt in enumerate(pts_ll):
    if extent[2]-1 <= pt[1] <= extent[3]+1 and extent[0]-1 <= pt[0] <= extent[1]+1:
        pts_ll_ext.append(pt)
        idx_ext_pts.append(i_pt)
pts_ll_ext = np.array(pts_ll_ext)


# plot regions and ISSRs
for i_reg, region in enumerate(sv.regions):
    if -1 not in region and len(region) > 0:
        polygon = [vts_ll[n] for n in region]
        polygon.append(vts_ll[region[0]])  # close the cell
        if any(extent[2]-1 <= v[1] <= extent[3]+1 and extent[0]-1 <= v[0] <= extent[1]+1 for v in polygon):
            if issr_arr[i_reg]:
                poly = mpatches.Polygon(polygon, closed=True, fill=False, ec=colors[0], hatch="////", alpha=0.8, transform=ccrs.PlateCarree(), zorder=2)
                ax.add_patch(poly)
            if temp_arr[i_reg]:
                poly = mpatches.Polygon(polygon, closed=True, fill=False, ec="gray", hatch="\\", alpha=0.8, transform=ccrs.PlateCarree(), zorder=2)
                ax.add_patch(poly)
            ax.plot(*zip(*polygon), "k", lw=1., transform=ccrs.PlateCarree(), zorder=3)

# plot regions per aircraft
ac_colors = ['#2271B2', '#3DB7E9', '#359B73', '#E69F00']
for i_ac, ac_id in enumerate(ac_ids):
    # plot edges used in summation
    for i in idx_ext_pts:
        for j in neighbors[i]:
            if j < i and mask_arr[i_ac, i] != mask_arr[i_ac, j]:
                v1, v2 = neighbors[i][j]['v1'], neighbors[i][j]['v2']
                start_vertex = vts_ll[v1]
                end_vertex = vts_ll[v2]
                
                # Adjust the line width and/or add transparency
                lw_adjust = 2*len(ac_ids)-1 - 2*i_ac  # Decrease line width slightly for each aircraft
                
                ax.plot([start_vertex[0], end_vertex[0]], [start_vertex[1], end_vertex[1]], 
                        c=ac_colors[i_ac], ls="-", lw=lw_adjust, 
                        transform=ccrs.Geodetic(), zorder=5 + i_ac)

fig.tight_layout()

## Create neighbours, perimeters, areas

These arrays/dictionaries are required for the main limiting factors analyses. In the following code, they can be created and saved to file.

To run, set the top-level project directory `project_dir` and, if not running the analysis on DKRZ Levante, set `dir_path` to the location of the ERA5 GRIB files. We recommend placing these files in `data/raw/`. Ensure that the file naming matches that of `t_file_path` in ln. 27 and 28. To save the pickled arrays, set `save_horizontal_neighbors` and/or `save_vertical_neighbors` to True.

In [ ]:
import xarray as xr
import numpy as np
import cf2cdm
import warnings
import pickle
from scipy.spatial import SphericalVoronoi
from itertools import permutations
from collections import defaultdict
from helper import lonlat_to_xyz, xyz_to_lonlat, haversine

# define directories
project_dir = ""  # set top-level directory location
processed_data_dir = project_dir + "data/processed/limfac/"

# load example GRIB file
dir_path = "/pool/data/ERA5/E5/pl/an/1H/"
date_str = "2024-02-11"  # random date for example
t_file_path = f"{dir_path}130/E5pl00_1H_{date_str}_130.grb"  # temperature file (130)
dsg = xr.open_dataset(t_file_path, engine="cfgrib", backend_kwargs={"indexpath": None})
with warnings.catch_warnings():  # ignoring UserWarning from cf2cdm when converting coordinate time -> time
    warnings.simplefilter('ignore')
    dsg = cf2cdm.translate_coords(dsg, cf2cdm.ECMWF)  # convert to ECMWF coordinates
dsg = dsg.isel(level=[18, 19, 20, 21, 22, 23, 24])  # selecting only the levels that are interesting

# define points in lonlat and cartesian
lat = dsg.latitude.values
lon = dsg.longitude.values
pts_ll = np.empty((len(lat), 2))
pts_ll[:, 0] = lon
pts_ll[:, 1] = lat
pts_ct = lonlat_to_xyz(lon, lat, radians=False)  # cartesian points

# spherical voronoi calculation
sv = SphericalVoronoi(pts_ct)
sv.sort_vertices_of_regions()  # this ensures that the vertices can be plotted properly
vts_ll = xyz_to_lonlat(sv.vertices, radians=False)  # lonlat vertices

# options
save_horizontal_neighbors = False
save_vertical_neighbors = False

In [ ]:
# create horizontal neighbours and perimeters
if save_horizontal_neighbors:
    # initialise neighbours dictionary
    neighbors = {i: {} for i in range(len(pts_ll))}

    # create a dictionary with each pair combination and the regions where the pairs occur
    pairs_dict = defaultdict(list)
    for index, lst in enumerate(sv.regions):
        for pair in permutations(lst, 2):
            reverse_pair = tuple(reversed(pair))
            if reverse_pair in pairs_dict:  # check if the reverse pair is already in dict
                continue
            pairs_dict[pair].append(index)

    # using pairs that occur in two different regions, calculate edge lengths
    shared_pairs = {pair: indexes for pair, indexes in pairs_dict.items() if len(indexes) > 1}
    for pair, indexes in shared_pairs.items():
        for i in indexes:
            for j in indexes:
                if i != j:
                    length = haversine(*vts_ll[pair[0]], *vts_ll[pair[1]])
                    if length != 0.:  # remove neighbors that are actually the same point (error in SphericalVoronoi method)
                        neighbors[i][j] = {"length": length, 'v1': pair[0], 'v2': pair[1]}
    
    # calculate perimeters
    perimeters = {region: sum(details['length'] for details in neighbors[region].values()) for region in neighbors}
    
    # save to file
    with open(processed_data_dir+"neighbors_grib.pickle", "wb") as f:
        pickle.dump(neighbors, f)
    with open(processed_data_dir+"perimeters_grib.pickle", "wb") as f:
        pickle.dump(perimeters, f)


# create vertical neighbours and areas
if save_vertical_neighbors:
    r = 6371.0  # [km] assumed radius of the Earth
    areas = sv.calculate_areas() * r ** 2  # [km2]
    
    # created flattened shape
    n_lvl = len(dsg.level.values)
    n_lat = len(dsg.latitude.values)
    flattened_mask = np.empty((1, n_lvl*n_lat))
    
    # initialize the dictionary to store connections
    vertical_neighbors_dict = {}
    
    # Loop through each "value" in the flattened mask
    for idx in range(flattened_mask.shape[1]):
        # Calculate the original level and value indices
        level = idx // n_lat
        value_idx = idx % n_lat
        # Create the connection dictionary for each value
        connections = {}
        if level > 0:  # Check if there is an upper level to connect
            upper_idx = (level - 1) * n_lat + value_idx
            connections[upper_idx] = {'area': areas[value_idx]}
        if level < n_lvl-1:  # Check if there is a lower level to connect
            lower_idx = (level + 1) * n_lat + value_idx
            connections[lower_idx] = {'area': areas[value_idx]}
        # Add the connections to the neighbors_dict
        vertical_neighbors_dict[idx] = connections
    
    # save to file
    with open(processed_data_dir+"vertical_neighbors_grib.pickle", "wb") as f:
        pickle.dump(vertical_neighbors_dict, f)